<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Zeitreihenanalyse - Neuronale Netze RNN - LSTM - Sinus
</b></font> </br></p>

---


# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
from pandas import DataFrame, read_csv, concat

from sklearn.model_selection import train_test_split

import keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense
from keras.models import load_model
from keras.utils import set_random_seed, plot_model

from tensorflow import keras
from tensorflow.config.experimental import enable_op_determinism
import tensorflow as tf

import plotly.express as px

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---

<p><font color='black' size="5">
Anwendungsfall
</font></p>

---   


Ein synthetischer Datensatz, der 10 Merkmale enthält und den Verlauf einer Sinuskurve m/o Trend hat. Funktionen alternativ ausführen!

In [ ]:
# @title
# @markdown   <p><font size="4" color='green'>  Synthetische Daten Sinus ohne Trend</font> </br></p>
# def create_dataframe(n_samples, cycles=5):
#     """
#     Erstellt einen DataFrame mit n_samples Datenpunkten, wobei die Zielgröße
#     eine Sinuskurve repräsentiert und die Merkmale M1-M10 darauf basierende Transformationen sind.

#     Parameters:
#     - n_samples: int, die Anzahl der zu generierenden Datenpunkte.

#     Returns:
#     - df: pandas DataFrame, der die generierten Daten enthält.
#     """
#     import numpy as np
#     import pandas as pd
#     # Basis für die Zielgröße
#     time = np.linspace(0, cycles * 2 * np.pi, n_samples)
#     target = np.sin(time)  # Zielgröße

#     # Erzeugen von Merkmalen, die direkt oder indirekt von der Zielgröße abgeleitet sind
#     features = {
#         'M1': np.sin(time),
#         'M2': np.cos(time),
#         'M3': np.sin(time)**2,
#         'M4': np.cos(time)**2,
#         'M5': np.sin(time + np.pi/4),
#         'M6': np.sin(time - np.pi/4),
#         'M7': np.cos(time + np.pi/4),
#         'M8': np.cos(time - np.pi/4),
#         'M9': np.sin(2*time),
#         'M10': np.cos(2*time)
#     }

#     # Erstellen des DataFrames
#     data = DataFrame(features)
#     data['target'] = target

#     return data

In [ ]:
# @title
# @markdown   <p><font size="4" color='green'>  Synthetische Daten Sinus mit Trend</font> </br></p>
def create_dataframe_trend(n_samples, cycles=5, trend_slope=0.01):
    """
    Erstellt einen DataFrame mit n_samples Datenpunkten, wobei die Zielgröße
    eine Sinuskurve repräsentiert und die Merkmale M1-M10 darauf basierende Transformationen sind.

    Parameters:
    - n_samples: int, die Anzahl der zu generierenden Datenpunkte.
    - trend_slope: float, die Steigung des linearen Trends.

    Returns:
    - df: pandas DataFrame, der die generierten Daten enthält.
    """
    import numpy as np
    import pandas as pd

    # Basis für die Zielgröße
    time = np.linspace(0, cycles * 2 * np.pi, n_samples)
    target = np.sin(time) + trend_slope * time  # Zielgröße mit linearem Trend

    # Erzeugen von Merkmalen, die direkt oder indirekt von der Zielgröße abgeleitet sind
    features = {
        "M1": np.sin(time),
        "M2": np.cos(time),
        "M3": np.sin(time) ** 2,
        "M4": np.cos(time) ** 2,
        "M5": np.sin(time + np.pi / 4),
        "M6": np.sin(time - np.pi / 4),
        "M7": np.cos(time + np.pi / 4),
        "M8": np.cos(time - np.pi / 4),
        "M9": np.sin(2 * time),
        "M10": np.cos(2 * time),
    }

    # Erstellen des DataFrames
    data = DataFrame(features)
    data["target"] = target

    return data

In [ ]:
# Erzeugen von x Datenpunkten mit y Zyklen
# Beispiel: 52 Woche x 5 Jahre → 260, 5
#
# data = create_dataframe(520, 10) # ohne Trend
data = create_dataframe_trend(520, 10, 0.001)  # mit Trend

<p><font color='black' size="5">
EDA (Exploratory Data Analysis)   
</font></p>

<p><font color='black' size="4">
Visualisierung der Daten
</font></p>

In [ ]:
fig = px.line(data, y="target", title="Grundprinzip", width=1000, height=500)
fig.show()

In [ ]:
target = data.pop("target")

# 2 | Prepare
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Train-Test-Split durchführen</br>
✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>


<p><font color='black' size="5">
Train-Test-Split
</font></p>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.2, shuffle=False
)
data_train.shape, data_test.shape, target_train.shape, target_test.shape

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

<p><font color='black' size="5">
Zufallszahl initialisieren
</font></p>

In [ ]:
set_random_seed(42)
enable_op_determinism()

<p><font color='black' size="5">
Übergreifende Parameter
</font></p

In [ ]:
past_steps = 52
future_steps = 52
num_features = 10


<p><font color='black' size="5">
Aufbau der Zeitscheiben
</font></p>

In [ ]:
generator = TimeseriesGenerator(
    data_train, target_train, length=past_steps, batch_size=future_steps
)

# # Beispiel erste Zeitscheibe
x, y = generator[0]
print(x.shape, y.shape)

<p><font color='black' size="5">
Modellaufbau
</font></p>

[Core layers](https://keras.io/api/layers/core_layers/)    
[Recurrent layers](https://keras.io/api/layers/recurrent_layers/)   
[Layer activation functions](https://keras.io/api/layers/activations/)   


In [ ]:
model = Sequential(name="Zeitreihenanalyse")
model.add(Input(shape=(past_steps, num_features)))
model.add(LSTM(64, activation="relu", return_sequences=True))
model.add(LSTM(32, activation="relu", return_sequences=False))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
# Visualisierung neuronales Netz
plot_model(
    model,
    to_file="nn_structure.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    dpi=100,
    expand_nested=True,
    show_layer_activations=True
)

In [ ]:
# Anzahl Parameter je Layer
for layer in model.layers:
    print(f"{layer.name}: {layer.count_params()} Parameter")


<p><font color='black' size="5">
Compile
</font></p>

In [ ]:
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

<p><font color='black' size="5">
Callbacks
</font></p>

In [ ]:
early = EarlyStopping(monitor="mae", patience=2)
check = ModelCheckpoint(filepath="model.keras", monitor="mae", save_best_only=True)

<p><font color='black' size="5">
Training
</font></p>

In [ ]:
model.fit(generator, epochs=5, batch_size=256, callbacks=[early, check])

In [ ]:
save_history = model.history.history

<p><font color='black' size="5">
Loss-Entwickung
</font></p>

In [ ]:
title_ = "Loss-Entwicklung"
px.line(
    y=save_history["loss"],
    title=title_,
    labels={"x": "Epochen", "y": "Loss-Wert"},
    width=800,
    height=400,
)

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Prognose
</font></p>

In [ ]:
### Bestes Modell laden
model = load_model("model.keras")

In [ ]:
# Einfache Vorhersage: Istdaten aus dem TestSet (Merkmale) werden werden zur Vorhersage eines Monats verwendet
prediction_generator = TimeseriesGenerator(
    data_test.values, target_test.values, length=past_steps, batch_size=future_steps
)

# Beispiel erste Zeitscheibe
x, y = prediction_generator[0]
print(x.shape, y.shape)

In [ ]:
# Vorhersagen machen
predictions = model.predict(prediction_generator)

In [ ]:
predictions.shape

<p><font color='black' size="5">
Vergleich Zeitreihen
</font></p>

In [ ]:
cube = DataFrame(target_test[future_steps:])  # Target ist x Perioden zu lang
cube["Predict"] = predictions

In [ ]:
fig = px.line(
    cube,
    y=["target", "Predict"],
    title="Zeitreihenanalyse - Target vs. Predict",
    width=1200,
    height=500,
)
fig.show()

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>

<p><font color='black' size="5">
Modell speichern
</font></p>

In [ ]:
model.save("model.keras")

In [ ]:
del model

<p><font color='black' size="5">
Modell einlesen
</font></p>

In [ ]:
new_model = load_model("model.keras")

In [ ]:
new_model.summary()

In [ ]:
new_model.get_weights()